#### FASE 1. Delimitación de los criterios de inclusión y extracción de las URL de las series que cumplan dichos criterios

Primero importo las librerías que voy a usar, e instalo el Driver

In [ ]:
#Para interactuar con la web, a través del navegador Chrome
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

#Para controlar que no ejecute un click hasta que la página se haya cargado
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#Para ir retrasando el web scraping y simular ser una persona
import time

#Para poder guardar mis Datos en un DataFrame
import pandas as pd

#Instalo el webdriver de Chrome
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service = service)

Defino los criterios de inclusión: quiero tomar, de la página de IMDb, las series producidas en Reino Unido en este milenio (del 1 de enero del 2000 en adelante), que sean de comedia, drama o suspense (y que no sean documentales, cortos, tertulias ni realities). Incluyo tanto series como miniseries, que hayan recibido al menos 100 votos en IMDb.

In [3]:
#Defino una función para tomar los enlaces de las series contenidas en cada URL

def get_links_series(genero):

    lista_links_series = []

    #Construyo la URL base, común para todas las series que quiero, donde ya están
    #especificados los criterios de inclusión. El único aspecto modificable es el género

    url = f'https://www.imdb.com/search/title/?title_type=tv_series,tv_miniseries&genres={genero},!documentary,!short,!talk-show,!reality&release_date=2000-01-01,&num_votes=100,&country_of_origin=GB'
    driver.get(url)

    #Acepto las cookies en caso de haberlas
    try:
        WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-testid = "accept-button"]'))).click()
    except:
        pass

    #Le pido que mientras haya opción de pinchar en Ver más, que lo haga 
    #(para que me despliegue todos los resultados de la página)
    while True:
        try:
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.sc-619d2eab-0 div span button'))).send_keys('\n')
        except:
            break  #Que salga del bucle cuando no haya más botones "Ver más"


    #Después extraigo cada link de las series contenidas en la URL principal y lo meto en una lista
    try:        
        bloque_series = driver.find_element(By.CSS_SELECTOR, 'div.ipc-page-grid__item > ul.ipc-metadata-list')
        series_indiv = bloque_series.find_elements(By.CSS_SELECTOR, 'li.ipc-metadata-list-summary-item > div > div > div > div.sc-b4e41383-3 > div.sc-1e00898e-0')
    except:
        pass

    try:
        for contenido in series_indiv:

            url_cada_peli = contenido.find_element(By.TAG_NAME, value='a').get_attribute('href')

            #Como cada link que me devuelve es del estilo https://www.imdb.com/title/tt7165904/?ref_=sr_t_1, 
            #y el último end point referencia a su posición concreta en la búsqueda, me tomo la licencia de eliminarlo, 
            # por si esto pudiera dar problemas en caso de que cambiara de orden

            url_cada_peli = str(url_cada_peli).split(sep = "?")[0]

            lista_links_series.append(url_cada_peli)

            time.sleep(1)
    except:
        pass
    
    #Paso la lista de URLs de cada género a DataFrame para luego poder unirlo con los DataFrame de los otros géneros
    df_links_series = pd.DataFrame(lista_links_series)

    #Hago copia del cada DataFrame en local
    df_links_series.to_csv(rf'df_links_GB_{genero}.csv')

    return df_links_series
    

Voy extrayendo los links de las series que cumplen con los criterios de inclusión

In [1]:
#Como lo único que cambia en la URL es el género, creo una lista con los posibles valores, 
#y la recorro con un bucle for

lista_generos = ['comedy', 'drama', 'thriller']
lista_dfs_links = []

#Para cada uno de los tres géneros, voy llamando a la función anterior y obteniendo un DataFrame
#que guardo dentro de una lista

for genero in lista_generos:

    df_links = get_links_series(genero)
    lista_dfs_links.append(df_links)

Monto mi DataFrame completo con todos los URLs

In [5]:
#Concateno los DataFrame de mi lista, eliminando los duplicados
df_links_completo = pd.concat(lista_dfs_links, ignore_index = True).drop_duplicates()

#Finalmente, guardo una copia en local de mi DataFrame con todas las URLs 
#(de los tres géneros, sin repetir)
df_links_completo.to_csv(r'df_links_GB_completo.csv')

#Compruebo que mi DataFrame ha guardado los datos correctamente
df_links_completo

0
0      https://www.imdb.com/title/tt7767422/
1      https://www.imdb.com/title/tt1888075/
2     https://www.imdb.com/title/tt11737520/
3     https://www.imdb.com/title/tt10590066/
4      https://www.imdb.com/title/tt8594324/
...                                      ...
2185   https://www.imdb.com/title/tt0312209/
2190   https://www.imdb.com/title/tt9015192/
2196   https://www.imdb.com/title/tt1722726/
2200   https://www.imdb.com/title/tt2296421/
2201   https://www.imdb.com/title/tt1517503/

[1707 rows x 1 columns]